# Objective: Artificial Neural Network Classification for Handwritten Digits Using MNIST
Here, I'm going to build an aritifical neural network classification model using PyTorch to classify handwritten digits using MNIST training data set.

## Import Libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt